In [1]:
import pandas as pd
from collections import OrderedDict
import os
import json

In [2]:
df = pd.DataFrame({'dataset': [], 'n_subsets': [], 'auc_pred': [], 'auc_att': [], 'seed': []})
for name in ['logic7', 'logic8', 'logic10', 'benzene', 'liver', 'ames']:
    metric = 'auc'
    path = '../%s' % name
    for dir_name in os.listdir(path):
        path_ = '%s/%s' % (path, dir_name)
        if os.path.isdir(path_):
            nsplits = dir_name.split('_')[-1]
            for seed in range(10):
                df.loc[len(df)] = name, int(nsplits), \
                    float(open('%s/%d/pred-%s' % (path_, seed, metric)).read()), \
                    float(open('%s/%d/att-auc' % (path_, seed)).read().split()[2][:-2]), \
                    seed
df.to_csv('../results/figure_data/hyperopt_mgk.csv', index=False)

In [3]:
df_gnn_pred = pd.DataFrame({'dataset': [], 'model': [], 'auc_pred': [], 'seed': []})
df_gnn_att = pd.DataFrame({'dataset': [], 'model': [], 'xai_tech': [], 'auc_att': [], 
                           'auc_att_full': [], 'seed': []})
for name in ['logic7', 'logic8', 'logic10', 'benzene', 'liver', 'ames']:
    metric = 'AUROC'
    for model in ['mpnn', 'gcn', 'gat', 'graphnet']:
        for seed in range(10):
            path = f'../gnn-ref/{name}-{model}-{seed}'
            df = pd.read_csv('%s/results.log' % path)
            df_gnn_pred.loc[len(df_gnn_pred)] = name, model, df[metric][0], seed
            for xai in ['Random', 'CAM', 'IG', 'GradInput', 'GradCAM-last', 
                        'GradCAM-all', 'SmoothGrad(GradInput)']:
                df_ = df[df.Technique == xai]
                df_gnn_att.loc[len(df_gnn_att)] = name, model, xai, df_['ATT AUROC'].iloc[0], \
                    float(json.loads(open('%s/%s.log' % (path, xai)).read())['att auc']), seed
df_gnn_pred.to_csv('../results/figure_data/pred_gnn.csv', index=False)
df_gnn_att.to_csv('../results/figure_data/att_gnn.csv', index=False)